# Projet de scraping CoinAfrique avec BeautifulSoup
Extraction des données de 4 catégories du site CoinAfrique en utilisant requests et BeautifulSoup, suivie du nettoyage, du traitement et de l’enregistrement des données dans une base de données SQLite centralisée.

In [15]:
pip install requests beautifulsoup4 pandas

Note: you may need to restart the kernel to use updated packages.


In [16]:
import sqlite3
import time
import pandas as pd
import requests
from bs4 import BeautifulSoup

HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/120.0.0.0 Safari/537.36"
    )
}

URL_DE_BASE = "https://sn.coinafrique.com/categorie/"

## Fonction de collecte de données réutilisable

In [17]:
def scrape_page(slug, page):
    """
    Scrape une seule page d'une catégorie CoinAfrique.

    Args:
        slug (str) : identifiant de la catégorie (ex: 'vetements-homme')
        page (int) : numéro de page

    Returns:
        list[dict] : liste d'annonces avec les clés
                     categorie, nom, prix, adresse, image_lien
    """
    url = f"{URL_DE_BASE}{slug}?page={page}"
    data = []
    try:
        response = requests.get(url, headers=HEADERS, timeout=20)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")
        containers = soup.find_all("div", class_="col s6 m4 l3")

        for container in containers:
            try:
                a_tag      = container.find("a", title=True)
                nom        = a_tag["title"].strip() if a_tag else ""

                prix_el    = container.find("p", class_="ad__card-price")
                prix       = prix_el.get_text(strip=True) if prix_el else ""

                adr_el     = container.find("p", class_="ad__card-location")
                adresse    = (
                    adr_el.get_text(separator=" ", strip=True)
                    .replace("location_on", "")
                    .strip()
                    if adr_el else ""
                )

                img_el     = container.find("img", class_="ad__card-img")
                image_lien = (
                    img_el.get("src") or img_el.get("data-src") or ""
                    if img_el else ""
                )

                data.append({
                    "categorie"  : slug,
                    "nom"        : nom,
                    "prix"       : prix,
                    "adresse"    : adresse,
                    "image_lien" : image_lien
                })
            except Exception:
                continue

    except Exception as e:
        print(f"  [!] Erreur page {page} ({slug}) : {e}")

    return data


def scrape_categorie(slug, nb_pages=9):
    """
    Scrape toutes les pages d'une catégorie et retourne un DataFrame.

    Args:
        slug     (str) : identifiant de la catégorie
        nb_pages (int) : nombre de pages à scraper

    Returns:
        pd.DataFrame
    """
    all_data = []
    for page in range(1, nb_pages + 1):
        rows = scrape_page(slug, page)
        all_data.extend(rows)
        print(f"  [{slug}] Page {page}/{nb_pages} — {len(rows)} annonces")
        time.sleep(1)
    return pd.DataFrame(all_data)

## PARTIE 1 : Simple Exploration 

In [18]:
CATEGORIES = [
    "chiens",
    "moutons",
    "poules_lapins_et_pigeons",
    "autres_animaux",
]

df_explore = pd.DataFrame()

for slug in CATEGORIES:
    print(f"\n--- {slug} ---")
    df_cat = scrape_categorie(slug, nb_pages=10)
    df_explore = pd.concat([df_explore, df_cat], ignore_index=True)

print(f"\nTotal collecté : {len(df_explore)} annonces")
df_explore.head()


--- chiens ---
  [chiens] Page 1/10 — 84 annonces
  [chiens] Page 2/10 — 84 annonces
  [chiens] Page 3/10 — 84 annonces
  [chiens] Page 4/10 — 84 annonces
  [chiens] Page 5/10 — 84 annonces
  [chiens] Page 6/10 — 84 annonces
  [chiens] Page 7/10 — 84 annonces
  [chiens] Page 8/10 — 84 annonces
  [chiens] Page 9/10 — 84 annonces
  [chiens] Page 10/10 — 84 annonces

--- moutons ---
  [moutons] Page 1/10 — 84 annonces
  [moutons] Page 2/10 — 84 annonces
  [moutons] Page 3/10 — 84 annonces
  [moutons] Page 4/10 — 84 annonces
  [moutons] Page 5/10 — 84 annonces
  [moutons] Page 6/10 — 84 annonces
  [moutons] Page 7/10 — 84 annonces
  [moutons] Page 8/10 — 84 annonces
  [moutons] Page 9/10 — 84 annonces
  [moutons] Page 10/10 — 84 annonces

--- poules_lapins_et_pigeons ---
  [!] Erreur page 1 (poules_lapins_et_pigeons) : 404 Client Error: Not Found for url: https://sn.coinafrique.com/categorie/poules_lapins_et_pigeons?page=1
  [poules_lapins_et_pigeons] Page 1/10 — 0 annonces
  [!] Erreur p

,categorie,nom,prix,adresse,image_lien
0,chiens,Chiots,150 000CFA,"Diamniadio, Sénégal",https://images.coinafrique.com/thumb_5782568_u...
1,chiens,Berger Belge Malinois,250 000CFA,"Sicap Liberté, Dakar, Sénégal",https://images.coinafrique.com/thumb_5777856_u...
2,chiens,Chien,100 000CFA,"Diamniadio, Sénégal",https://images.coinafrique.com/thumb_5769939_u...
3,chiens,Chiot Bouledogues français,Prix sur demande,"Point E, Dakar, Sénégal",https://images.coinafrique.com/thumb_5757166_u...
4,chiens,Chiots Berger Allemand,150 000CFA,"Guediawaye, Dakar, Sénégal",https://images.coinafrique.com/thumb_5751732_u...


## PARTIE 2 : Nettoyage et vérification des données

In [19]:
# Taille du dataset
print("Shape :", df_explore.shape)

# Types des variables
print("\nTypes :")
print(df_explore.dtypes)

# Valeurs manquantes
print("\nValeurs manquantes :")
print(df_explore.isna().sum())

# Doublons
print("\nDoublons :", df_explore.duplicated().sum())

# Répartition par catégorie
print("\nRépartition par catégorie :")
print(df_explore["categorie"].value_counts())

Shape : (1680, 5)

Types :
categorie     object
nom           object
prix          object
adresse       object
image_lien    object
dtype: object

Valeurs manquantes :
categorie     0
nom           0
prix          0
adresse       0
image_lien    0
dtype: int64

Doublons : 0

Répartition par catégorie :
categorie
chiens     840
moutons    840
Name: count, dtype: int64


## Nous voyons directement que nous n'avons aucunement pas de doublons ni de données manquantes ni moins des valeurs aberrantes

## PARTIE 3 : Extraction complète des données et enregistrement dans une base SQLite

In [ ]:
import sqlite3
import pandas as pd

# Connexion à la base unifiée
conn = sqlite3.connect("coinafrique_bs4.db")

conn.execute("""
    CREATE TABLE IF NOT EXISTS annonces (
        categorie  TEXT,
        nom        TEXT,
        prix       TEXT,
        adresse    TEXT,
        image_lien TEXT
    )
""")
conn.commit()

# Scraper et insérer chaque catégorie
for slug in CATEGORIES:
    print(f"\n--- Catégorie : {slug} ---")

    df_cat = scrape_categorie(slug, nb_pages=9)

    # 🔎 Vérification si DataFrame vide
    if df_cat.empty:
        print("  Aucune donnée récupérée.")
        continue

    # 🔎 Vérification des colonnes
    colonnes_attendues = ["categorie", "nom", "prix", "adresse", "image_lien"]

    for col in colonnes_attendues:
        if col not in df_cat.columns:
            df_cat[col] = ""  # on crée la colonne manquante vide

    #  Nettoyage sécurisé
    df_cat = df_cat[df_cat["nom"].astype(str).str.strip() != ""]
    df_cat = df_cat[df_cat["prix"].astype(str).str.strip() != ""]
    df_cat = df_cat.drop_duplicates()

    #  Insertion dans SQLite
    df_cat.to_sql("annonces", conn, if_exists="append", index=False)

    print(f"  => {len(df_cat)} annonces insérées")

print("\n Scraping terminé.")
conn.close()


--- Catégorie : chiens ---
  [chiens] Page 1/9 — 84 annonces
  [chiens] Page 2/9 — 84 annonces
  [chiens] Page 3/9 — 84 annonces
  [chiens] Page 4/9 — 84 annonces
  [chiens] Page 5/9 — 84 annonces
  [chiens] Page 6/9 — 84 annonces
  [chiens] Page 7/9 — 84 annonces
  [chiens] Page 8/9 — 84 annonces
  [chiens] Page 9/9 — 84 annonces
  => 756 annonces insérées

--- Catégorie : moutons ---
  [moutons] Page 1/9 — 84 annonces
  [moutons] Page 2/9 — 84 annonces
  [moutons] Page 3/9 — 84 annonces
  [moutons] Page 4/9 — 84 annonces
  [moutons] Page 5/9 — 84 annonces
  [moutons] Page 6/9 — 84 annonces
  [moutons] Page 7/9 — 84 annonces
  [moutons] Page 8/9 — 84 annonces
  [moutons] Page 9/9 — 84 annonces
  => 756 annonces insérées

--- Catégorie : poules_lapins_et_pigeons ---
  [!] Erreur page 1 (poules_lapins_et_pigeons) : 404 Client Error: Not Found for url: https://sn.coinafrique.com/categorie/poules_lapins_et_pigeons?page=1
  [poules_lapins_et_pigeons] Page 1/9 — 0 annonces
  [!] Erreur pag

## PARTIE 4 : Vérification de la base de données

In [ ]:
import sqlite3
import pandas as pd

#  Reconnexion à la base
conn = sqlite3.connect("coinafrique_bs4.db")

df_all = pd.read_sql_query("SELECT * FROM annonces", conn)

print("Shape :", df_all.shape)

print("\nRépartition par catégorie :")
print(df_all["categorie"].value_counts())

print("\nValeurs manquantes :")
print(df_all.isna().sum())

print("\nDoublons :", df_all.duplicated().sum())

conn.close()

df_all.head()

Shape : (2688, 5)

Répartition par catégorie :
categorie
moutons    1428
chiens     1260
Name: count, dtype: int64

Valeurs manquantes :
categorie     0
nom           0
prix          0
adresse       0
image_lien    0
dtype: int64

Doublons : 1176


,categorie,nom,prix,adresse,image_lien
0,chiens,Chiot Husky,350 000CFA,"Pikine, Sénégal",https://images.coinafrique.com/thumb_5456601_u...
1,chiens,Chiots malinois de parents importés de bordeau...,250 000CFA,"Thies, Sénégal",https://images.coinafrique.com/thumb_5455656_u...
2,chiens,Chiot Rottweiler,290 000CFA,"Pikine, Sénégal",https://images.coinafrique.com/thumb_5453153_u...
3,chiens,Chiot Berger Allemand,120 000CFA,"Lac rose, Sénégal",https://images.coinafrique.com/thumb_5443216_u...
4,chiens,Chiot Berger allemand,150 000CFA,"Pikine, Sénégal",https://images.coinafrique.com/thumb_5439144_u...
